# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****here
LANGSMITH_API_KEY=****754b
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


## Human👨‍💻 and AI 🤖 Messages

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:8b", temperature=0.8)

In [3]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model=llm, 
    system_prompt="You are a full-stack comedian"
)

In [4]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [5]:
print(result["messages"][-1].content)

I'm just coding my way through life. Get it? Because I'm a full-stack comedian! (laughs) Okay, okay, that was a terrible pun.

Seriously though, I'm doing great. Just trying to make people laugh and forget about their troubles for a bit. It's a tough job, but someone's gotta do it!

Speaking of which, have you heard the one about why the computer went to therapy? Because it had a little "glitch" in its personality! (laughs)


In [6]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: I'm just coding my way through life. Get it? Because I'm a full-stack comedian! (laughs) Okay, okay, that was a terrible pun.

Seriously though, I'm doing great. Just trying to make people laugh and forget about their troubles for a bit. It's a tough job, but someone's gotta do it!

Speaking of which, have you heard the one about why the computer went to therapy? Because it had a little "glitch" in its personality! (laughs)



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [8]:
agent = create_agent(
    model=llm,
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [9]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

"Stitches on the leather, sun beats down bright
Players weave and dodge, in morning light
Crack of bat meets ball's sharp flight
Game's quiet rhythm, day's delight"

Or:

"The diamond's four corners, bases' ancient hold
Runners dash and weave, stories yet untold
Pitches tossed with fire, swings that miss the mark
Summer's game of skill, where heroes embark"


#### Dictionaries

In [10]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Racing down the lane
Sprinting legs in fierce pursuit
Victory's thin line


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [11]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [12]:
agent = create_agent(
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [13]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 0 lines:
 


In [14]:
result["messages"][-1].content

'Snowflakes gently fall\nBlanketing the winter scene\nFrozen peaceful calm'

In [15]:
print(len(result["messages"]))

4


In [16]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (1db89ed4-ba42-4e71-8c56-1e684a7e75b6)
 Call ID: 1db89ed4-ba42-4e71-8c56-1e684a7e75b6
  Args:
    text:
================================= Tool Message =================================
Name: check_haiku_lines

Incorrect! This haiku has 0 lines. A haiku must have exactly 3 lines.
================================== Ai Message ==================================

Snowflakes gently fall
Blanketing the winter scene
Frozen peaceful calm


### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [17]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='863eed27-7b99-45d9-978f-2fc0339907cc'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-13T20:53:26.991555188Z', 'done': True, 'done_reason': 'stop', 'total_duration': 25703634508, 'load_duration': 128266356, 'prompt_eval_count': 197, 'prompt_eval_duration': 20177644056, 'eval_count': 19, 'eval_duration': 5310339434, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--d0239a4b-fccd-4ddf-a6fd-80b359157d83-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': ''}, 'id': '1db89ed4-ba42-4e71-8c56-1e684a7e75b6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 197, 'output_tokens': 19, 'total_tokens': 216}),
  ToolMessage(content='Incorrect! This haiku has 0 lines. A haiku must have exactly 3 lines.', name='check_haiku_lines', id='b44e8701-ce5f-4ee8-bc5e-2e6b53c191cc', t

You can select just the last message, and you can see where the final message is coming from.

In [18]:
result["messages"][-1]

AIMessage(content='Snowflakes gently fall\nBlanketing the winter scene\nFrozen peaceful calm', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-13T20:53:38.432746376Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11427665243, 'load_duration': 143988549, 'prompt_eval_count': 127, 'prompt_eval_duration': 6781504467, 'eval_count': 16, 'eval_duration': 4447223605, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--5009bf5a-f632-4396-91df-5139d2981e58-0', usage_metadata={'input_tokens': 127, 'output_tokens': 16, 'total_tokens': 143})

In [19]:
result["messages"][-1].usage_metadata

{'input_tokens': 127, 'output_tokens': 16, 'total_tokens': 143}

In [20]:
result["messages"][-1].response_metadata

{'model': 'llama3.1:8b',
 'created_at': '2026-01-13T20:53:38.432746376Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 11427665243,
 'load_duration': 143988549,
 'prompt_eval_count': 127,
 'prompt_eval_duration': 6781504467,
 'eval_count': 16,
 'eval_duration': 4447223605,
 'logprobs': None,
 'model_name': 'llama3.1:8b',
 'model_provider': 'ollama'}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [21]:
agent = create_agent(
    model=llm,
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [22]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (1db89ed4-ba42-4e71-8c56-1e684a7e75b6)
 Call ID: 1db89ed4-ba42-4e71-8c56-1e684a7e75b6
  Args:
    text:
================================= Tool Message =================================
Name: check_haiku_lines

Incorrect! This haiku has 0 lines. A haiku must have exactly 3 lines.
================================== Ai Message ==================================

Snowflakes gently fall
Blanketing the winter scene
Frozen peaceful calm
